#### This notebook has been tested with SageMaker Studio Notebooks, with Data science 3.0 and Python 3 environment

## Set up your environment

To complete this example, we must install several Python dependencies.

In [194]:
!pip install -q langchain==0.0.266

To deploy the LLM, we leverage Amazon SageMaker Jumpstart, following the blog post [Llama 2 foundation models from Meta are now available in Amazon SageMaker JumpStart](https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/). Please provide the name of your endpoint after deployment for confirmation. Also, ensure that your endpoint is located in the same region as the notebook. 

Please make sure you followed the blog and to deploy the correct model.

<img src="./imgs/jumpstart.png" alt="jumpstart" style="width: 400px;"/>


In [195]:
import boto3
aws_region = boto3.Session().region_name
print(aws_region)
endpoint_name = "jumpstart-dft-meta-textgeneration-llama-2-7b-f"
print(endpoint_name)

us-east-1
jumpstart-dft-meta-textgeneration-llama-2-7b-f


## Creat your prompts

The input passed to a large language model is referred to as a "prompt". A PromptTemplate handles how this input text is dynamically formed. LangChain offers various utilities and abstractions to simplify the process of building, modifying, and interacting with prompts. 

This example demonstrates a prompt that incorporates user preferences, allowing the large language model to have a context of what attributes are most relevant to the user and generating an email based on the context.

In [196]:
from langchain import PromptTemplate

prompt_template = """

Our company, "Classic Cinema," frequently promotes movies that we aim to recommend to our customers. This month, we have several popular movies on promotion.

As an AI agent, you are tasked to assist "Classic Cinema" in crafting an email campaign to recommend relevant movies to users. The recommendations should adhere to several guidelines, including contextual relevance, ensuring the recommendations are strictly from our promotional movie list. Additionally, the recommendations should align with user preferences, suggesting items that are relevant and in harmony with the user's preferred categories. You are to provide precisely three top recommended movies. Finally, please draft the email to reflect the tone of the user's preferred categories. The email should not exceed 200 words.

The recommended movies should be sourced from this contextual relevance movie list: 
{promotion_movie_list}.

The user has expressed interest in {user_preference}. 

Please ensure the recommendations are relevant, and the tone of the email reflects the tastes of those interested in the {user_preference} movie category.

Ensure the letter appeals to those interested in the {user_preference} movie category. 
Email body should start with Dear [Name], and keep the email campaign within a 200 word limit. 


"""
prompt = PromptTemplate.from_template(prompt_template)

## Integrate the PromptTemplate with FeatureStore

The missing part from previous prompt is the user preference.

Here's a demonstration of integrating Amazon SageMaker FeatureStore to create a prompt to describe user preferences:

1. Fetch the user preference from the FeatureStore:
2. Rank user preferences
3. Generate a formatted prompt:



In [197]:
from langchain.prompts import StringPromptTemplate
import pandas as pd

class FeatureStorePromptTemplate(StringPromptTemplate):
    
    feature_group_name = 'user-profile-feature-group'
    promotion_list_path = './movie-meta-data/promotion_movie.csv'
    
    
    def format(self, **kwargs) -> str:
        user_id = kwargs.pop("user_id")
        feature_record = self.fetch_user_preference_from_feature_store(user_id)
        user_preference = self.rank_user_preference(feature_record)
        
        
        kwargs["promotion_movie_list"] = self.read_promotion_list()
        kwargs["user_preference"] = user_preference
        return prompt.format(**kwargs)
    
    
    def fetch_user_preference_from_feature_store(self, user_id):
        
        boto_session = boto3.Session()
        featurestore_runtime_client = boto_session.client('sagemaker-featurestore-runtime')
        feature_record = featurestore_runtime_client.get_record(FeatureGroupName=self.feature_group_name, 
                                                        RecordIdentifierValueAsString=str(user_id))

        return feature_record['Record']
    
    
    def rank_user_preference(self, data) -> str:
        # Transform the data into a dictionary
        data_dict = {item['FeatureName']: item['ValueAsString'] for item in data}

        # Create a pandas DataFrame from the dictionary
        df = pd.DataFrame([data_dict])
        # Drop the last column
        df = df.drop("EventTime", axis=1)
        
        # Set 'User_ID' as the index
        df.set_index('User_ID', inplace=True)
        # Convert the data to floats
        df = df.astype(float)
        
        # Transpose the dataframe, then apply nlargest() on each column
        top_categories = df.transpose().apply(lambda x: x.nlargest(3).index.tolist(), axis=0)

        # Join the top 3 categories into a single string for each user
        top_categories = top_categories.apply(', '.join, axis=1)

        # Convert the series to a dataframe
        top_categories_df = top_categories.to_frame(name='Top_3_Categories')        
        
        nested_list = top_categories_df.values.tolist()
        comma_separated_categories = ', '.join([item for sublist in nested_list for item in sublist])

        return comma_separated_categories
        
        
    def read_promotion_list(self,) -> str:
        df = pd.read_csv(self.promotion_list_path)
        df = df.drop('ID', axis=1)
        
        output_string = ''
        for _, row in df.iterrows():
            row_str = ' '.join(f'{col}: {val}' for col, val in zip(df.columns, row))
            output_string += row_str + '\n'

        return output_string

In [198]:
prompt_template = FeatureStorePromptTemplate(input_variables=["user_id"])

In [199]:
print(prompt_template.format(user_id=4))



Our company, "Classic Cinema," frequently promotes movies that we aim to recommend to our customers. This month, we have several popular movies on promotion.

As an AI agent, you are tasked to assist "Classic Cinema" in crafting an email campaign to recommend relevant movies to users. The recommendations should adhere to several guidelines, including contextual relevance, ensuring the recommendations are strictly from our promotional movie list. Additionally, the recommendations should align with user preferences, suggesting items that are relevant and in harmony with the user's preferred categories. You are to provide precisely three top recommended movies. Finally, please draft the email to reflect the tone of the user's preferred categories. The email should not exceed 200 words.

The recommended movies should be sourced from this contextual relevance movie list: 
Movie Title: The Shawshank Redemption Description: Wrongfully accused Andy Dufresne is sentenced to life in Shawshank 

## End to End Demo with Langchain

Now that we have a well-crafted prompt to communicate with the large language model using the FeatureStore prompt, it's time to invoke the model and generate an email.


In [200]:
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json

class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs" : [[{"role" : "system",
        "content" : "You are a kind robot."},
        {"role" : "user", "content" : prompt}]],
        "parameters" : {**model_kwargs}})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generation"]["content"]
    

parameters = {
    "max_new_tokens": 1000
}

content_handler = ContentHandler()

sm_llm = SagemakerEndpoint(
    endpoint_name = endpoint_name, 
    region_name = aws_region, 
    model_kwargs = parameters,
    endpoint_kwargs={"CustomAttributes": 'accept_eula=true'},
    content_handler = content_handler,
)

In [202]:
from langchain.chains import LLMChain

llmchain = LLMChain(llm=sm_llm, prompt=prompt_template)
email_content = llmchain.run({
    'user_id': 4,
    })

# OUTPUT
# PodConneXion

In [203]:
print(email_content)

 Subject: Explore the Best of Sci-Fi, Adventure, and War Movies with Classic Cinema!

Dear [Name],

At Classic Cinema, we understand your love for movies that transport you to new worlds, fill your hearts with adventure, and leave you on the edge of your seat. That's why we're excited to share our top recommended movies from the Sci-Fi, Adventure, and War genres.

1. Interstellar (2014) - A visually stunning and thought-provoking journey through space, exploring the vastness of the universe and the power of love.
2. The Lord of the Rings: The Fellowship of the Ring (2001) - Join Frodo and his companions on an epic quest to destroy the One Ring and save Middle-earth from the darkness of Sauron's rule.
3. Saving Private Ryan (1998) - Experience the intense and emotional story of a group of soldiers on a dangerous mission to find and rescue a paratrooper whose brothers have been killed in action.

These movies are sure to ignite your passion for adventure, exploration, and the human spiri